# Answering Business Questions using SQL

In this project, we're going to answer business questions.

## chinook database schema diagram -
![schema](chinook_schema.jpg)

In [1]:
# Connect our Jupyter Notebook to our database file:

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

In [3]:
# Query database to get a list of all tables and views in our database

In [4]:
%%sql
SELECT name, type
FROM sqlite_master 
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

In [5]:
# Query that returns no. of tracks sold per genre in USA - both in absolute nos. and percentages

In [6]:
%%sql
CREATE VIEW IF NOT EXISTS genres_sold_usa AS 
    SELECT g.name, c.country, COUNT(g.name) AS nos_sold
    FROM customer AS c INNER JOIN invoice AS i ON c.customer_id = i.customer_id
         INNER JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
         INNER JOIN track AS t ON il.track_id = t.track_id 
         INNER JOIN genre as g ON t.genre_id = g.genre_id
    GROUP BY g.name, c.country
    HAVING c.country = "USA"
    ORDER BY nos_sold DESC;
    
SELECT * FROM genres_sold_usa;

 * sqlite:///chinook.db
Done.
Done.


name,country,nos_sold
Rock,USA,561
Alternative & Punk,USA,130
Metal,USA,124
R&B/Soul,USA,53
Blues,USA,36
Alternative,USA,35
Latin,USA,22
Pop,USA,22
Hip Hop/Rap,USA,20
Jazz,USA,14


In [7]:
# Show each genre sold in usa - both absolute nos and percentage

In [8]:
%%sql

SELECT name AS genre_name, nos_sold,
       (CAST(nos_sold AS FLOAT) / (SELECT SUM(nos_sold) FROM genres_sold_usa)) * 100 AS percent_share
FROM genres_sold_usa;

 * sqlite:///chinook.db
Done.


genre_name,nos_sold,percent_share
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


## Analyzing Employee Sales Performance

In [9]:
%%sql

SELECT e.employee_id, e.hire_date, SUM(i.total) AS total_sales
FROM invoice AS i INNER JOIN customer AS c ON i.customer_id = c.customer_id
INNER JOIN employee AS e ON c.support_rep_id = e.employee_id
GROUP BY e.employee_id
ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


employee_id,hire_date,total_sales
3,2017-04-01 00:00:00,1731.5100000000039
4,2017-05-03 00:00:00,1584.0000000000034
5,2017-10-17 00:00:00,1393.920000000002


While there is a difference in sales, the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

In [10]:
%%sql
SELECT COUNT(DISTINCT country) AS total_countries FROM customer;

 * sqlite:///chinook.db
Done.


total_countries
24


In [11]:
%%sql
CREATE VIEW IF NOT EXISTS country_sales AS
    SELECT c.country, COUNT(DISTINCT c.customer_id) AS total_customers,
           SUM(i.total) AS total_sales,
           (CAST(SUM(i.total) AS FLOAT) / COUNT(c.customer_id)) AS avg_order_value
    FROM invoice AS i INNER JOIN customer AS c ON i.customer_id = c.customer_id
    GROUP BY c.country
    ORDER BY total_sales DESC;
    
SELECT * FROM country_sales;

 * sqlite:///chinook.db
Done.
Done.


country,total_customers,total_sales,avg_order_value
USA,13,1040.4899999999998,7.942671755725189
Canada,8,535.5900000000001,7.047236842105265
Brazil,5,427.68000000000006,7.0111475409836075
France,5,389.0699999999999,7.781399999999998
Germany,4,334.62,8.161463414634147
Czech Republic,2,273.24000000000007,9.108000000000002
United Kingdom,3,245.52,8.768571428571429
Portugal,2,185.13000000000002,6.383793103448276
India,2,183.14999999999998,8.72142857142857
Ireland,1,114.83999999999997,8.833846153846151


## Albums vs Individual Tracks

In [12]:
%%sql
CREATE VIEW IF NOT EXISTS album_tracks AS
    
     SELECT album_id, track_id
     FROM track;
    
SELECT * FROM album_tracks LIMIT 20;

 * sqlite:///chinook.db
Done.
Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14


In [13]:
%%sql
CREATE VIEW IF NOT EXISTS invoice_tracks AS

    SELECT i.invoice_id, t.track_id 
    FROM invoice AS i INNER JOIN invoice_line AS il ON i.invoice_id = il.invoice_id
    INNER JOIN track AS t ON il.track_id = t.track_id;
    
SELECT * FROM invoice_tracks LIMIT 20;

 * sqlite:///chinook.db
Done.
Done.


invoice_id,track_id
1,1158
1,1159
1,1160
1,1161
1,1162
1,1163
1,1164
1,1165
1,1166
1,1167


In [14]:
%%sql
WITH full_album_invoices AS 
    (
     SELECT i.invoice_id
     FROM invoice_tracks AS i INNER JOIN album_tracks AS a ON i.track_id = a.track_id
     GROUP BY i.invoice_id 
     HAVING COUNT(DISTINCT a.album_id) = 1 
            AND 
            COUNT(i.track_id) = (SELECT COUNT(track_id) FROM album_tracks WHERE album_id = a.album_id)
    )
            
SELECT COUNT(f.invoice_id) AS invoices_with_full_album_purchase,
       (SELECT COUNT (*) FROM invoice) AS total_invoices,
       (CAST(COUNT(f.invoice_id) AS FLOAT) / (SELECT COUNT (*) FROM invoice)) * 100 
            AS percent_full_album_sales
FROM full_album_invoices AS f;

 * sqlite:///chinook.db
Done.


invoices_with_full_album_purchase,total_invoices,percent_full_album_sales
114,614,18.566775244299674


Full album purchases account for only 18.6% of purchases, so recommended to purchase only select tracks from an album rather than an entire album.